# Chat Completion API

In [1]:
!pip install openai

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google colab - secret탭 - `OPENAI_API_KEY` 등록

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [4]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [12]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [27]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕 내 이름은 차은우야"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "안녕하세요, 차은우님! 만나서 반가워요 😊  \n오늘은 어떤 도움이 필요하신가요?  \n편하게 궁금한 점이나 하고 싶은 이야기를 말씀해 주세요!"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=7960,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [28]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반가워요 😊  \n오늘은 어떤 도움이 필요하신가요?  \n편하게 궁금한 점이나 하고 싶은 이야기를 말씀해 주세요!'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력 데이터/예시
4. Output Indicator 출력지시

## 기사 제목 교정 예제

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용

In [21]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요'
title_before = '겁나 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완곡하고 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

# Steps (작업 절차)
1. 기사 제목을 읽고 주요 내용을 이해하세요.
2. 제목이 전달하고자 하는 메시지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결하게 정리하세요.

# Output Format
기사의 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.
- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

# Examples (2-shot-prompting)
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차 회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차 회사들"

# Extra Instructions
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메시지를 잃어버리면 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 해당 표현을 완화하세요.

"""

user_message = f"""
다음 기사 제목을 짧게 교정해 주세요.

{title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message.content)

- 원래 제목: "테이의 FM 개꿀 라디오 방송에 주목해주세요"
- 교정 제목: "테이의 FM 라디오 방송에 주목하세요"


In [22]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):

  # 역할/페르소나/지시사항
  system_message = """
  기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정작업을 수행해 주세요.

  - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
  - 독자의 관심을 끌 수 있도록 간결하면서 임팩트 있는 표현을 사용하세요.
  - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완곡하고 중립적인 어조로 수정하세요.
  - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

  # Steps (작업 절차)
  1. 기사 제목을 읽고 주요 내용을 이해하세요.
  2. 제목이 전달하고자 하는 메시지를 명확하게 반영하는지 검토하세요.
  3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
  4. 제목이 자연스럽고 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결하게 정리하세요.

  # Output Format
  기사의 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.
  - 원래 제목: [기사 원래 제목]
  - 교정 제목: [교정 기사 제목]

  # Examples (2-shot-prompting)
  - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
  - 교정 제목: "서울 대형 화재, 수백명 대피"

  - 원래 제목: "전기자동차 판매량 급감에 내연차 회사들이 즐거워하는 중입니다."
  - 교정 제목: "전기차 판매량 급감에 웃는 내연차 회사들"

  # Extra Instructions
  - 제목이 너무 길면, 간결하게 줄이되, 핵심 메시지를 잃어버리면 안됩니다.
  - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
  - 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 해당 표현을 완화하세요.

  """

  user_message = f"""
  다음 기사 제목을 짧게 교정해 주세요.

  {title_before}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: "주말 미친 폭우 예상, 모두들 무사하시길~"
- 교정 제목: "주말 폭우 예상, 안전 유의하세요"


## 영문 텍스트에 anki 단어장 만들기